In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import json
import sys

In [51]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 100.0.4896
[WDM] - Get LATEST chromedriver version for 100.0.4896 google-chrome
[WDM] - Driver [C:\Users\GuoshengCai\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [77]:
def find_department_pages(driver, departments_url, no_pages, css_selector, nth_element=None):
    driver.get(departments_url)
    items = []
    for i in range(no_pages):
        if i != 0:
            to_click = driver.find_element(By.LINK_TEXT, 'Next')
            to_click.click()
        elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
        if nth_element is not None:
            elements = elements[nth_element].find_elements(By.CSS_SELECTOR,"*")
        for element in elements:
            try:
                tmp = element.find_element(By.TAG_NAME, 'a')
                item = {'department': tmp.get_attribute('textContent'), 'url': tmp.get_attribute('href')}
                items.append(item)
            except:
                pass
    return items

In [29]:
def find_faculty_page(driver, url):
    driver.get(url)
    elements = driver.find_elements(By.XPATH, 
    '//a[contains(@href,"faculty") or contains(@href,"profile") or contains(@href,"people") or contains(@href,"directory") or contains(@href,"staff")]')
    if len(elements) == 0:
        lk = None
    else:
        lk = select_lk(elements)
    return lk

In [30]:
def select_lk(elements):
    priority_dict = {'faculty directory': 7, 'faculty list': 6, 'faculty': 5, 'people': 4,'directory': 3,
                     'profiles': 2, 'staff': 1}
    lk = None
    priority = -1
    for element in elements:
        s = element.get_attribute("innerText").lower()
        curr_priority = 0
        for t in priority_dict:
            if t == s:
                curr_priority = priority_dict.get(t)
                break
        if curr_priority > priority:
            priority = curr_priority
            lk = element.get_attribute('href')
    return lk

# Columbia

In [24]:
departments_url = "https://www.columbia.edu/content/academics/departments"
no_pages = 5
css_selector = '.dynamic-grid-listing-item.grid-item.angular-animate.ng-trans.ng-trans-fade-down.ng-scope'
columbia_departments = find_department_pages(driver, departments_url, no_pages, css_selector)

In [91]:
for item in columbia_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [93]:
with open(sys.path[0] + '/../data/columbia/columbia_directories_updated.json', 'w') as f:
    json.dump(columbia_departments, f)

# Dartmouth

In [123]:
departments_url = "https://home.dartmouth.edu/academics/departments-programs"
no_pages = 1
nth_element = 2
css_selector = '.clearfix.text-formatted.field.field--name-body.field--type-text-with-summary.field--label-hidden.field__item'
dartmouth_departments = find_department_pages(driver, departments_url, no_pages, css_selector, nth_element)

In [131]:
# https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python
def find_faculty_page_dartmouth(driver, url):
    driver.get(url)
    try:
        element = driver.find_element(By.XPATH, 
        '//a[contains(@href,"people")]') 
        lk = element.get_attribute('href')
        driver.get(lk)
        # https://stackoverflow.com/questions/26958193/how-to-select-any-element-from-the-web-element-with-display-none-attribute-us
        driver.execute_script("document.getElementById('edit-field-people-local-tags-tid').style.display='inline-block';")
        element = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.ID, 'edit-field-people-local-tags-tid')))
        select = Select(element)
        try:
            select.select_by_visible_text('Faculty')
        except:
            pass

        try:
            select.select_by_visible_text('faculty')
        except:
            select.select_by_index(1)

        to_click = driver.find_element(By.ID, 'edit-submit-people')
        to_click.click()
        currentURL = driver.current_url
        return currentURL
    except:
        return None

In [132]:
for item in dartmouth_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page_dartmouth(driver, url)})

In [134]:
with open(sys.path[0] + '/../data/dartmouth/dartmouth_directories.json', 'w') as f:
    json.dump(dartmouth_departments, f)

# Yale

In [26]:
departments_url = "https://www.yale.edu/academics/departments-programs"
no_pages = 1
css_selector = '.department_item_heading'
yale_departments = find_department_pages(driver, departments_url, no_pages, css_selector)

In [31]:
for item in yale_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [34]:
with open(sys.path[0] + '/../data/yale/yale_directories.json', 'w') as f:
    json.dump(yale_departments, f)

# Princeton

In [69]:
def find_department_pages_princeton(driver):
    driver.get('https://www.princeton.edu/academics/areas-of-study')
    items = []
    elements = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Academic Units')]")
    parents = []
    for element in elements:
        element = element.find_element(By.XPATH, "..")
        try:
            tmp = element.find_element(By.TAG_NAME, 'a')
            item = {'department': tmp.get_attribute('textContent'), 'url': tmp.get_attribute('href')}
            items.append(item)
        except:
            pass
    return items

In [71]:
princeton_departments = find_department_pages_princeton(driver)

In [72]:
for item in princeton_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [73]:
with open(sys.path[0] + '/../data/princeton/princeton_directories.json', 'w') as f:
    json.dump(princeton_departments, f)

# Brown

In [78]:
departments_url = "https://www.brown.edu/academics/departments"
no_pages = 1
css_selector = '.news-items'
brown_departments = find_department_pages(driver, departments_url, no_pages, css_selector)

In [79]:
for item in brown_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [80]:
with open(sys.path[0] + '/../data/brown/brown_directories.json', 'w') as f:
    json.dump(brown_departments, f)